In [1]:
import numpy as np
common_channels = ['FP1','FP2','F7','F3','FZ','F4','F8','T3','C3','CZ','C4','T4','T5','P3','PZ','P4','T6','O1','O2']
montage_reference = [(0,2),(2,7),(7,12),(12,17),(1,6),(6,11),(11,16),(16,18),(7,8),(8,9),(9,10),(10,11),(0,3),(3,8),(8,13),(13,17),(1,5),(5,10),(10,15),(15,18)]
for (i,j) in montage_reference:
    print(common_channels[i],'-',common_channels[j])
def convert_montage(x):
    montage_reference = [(0,2),(2,7),(7,12),(12,17),(1,6),(6,11),(11,16),(16,18),(7,8),(8,9),(9,10),(10,11),(0,3),(3,8),(8,13),(13,17),(1,5),(5,10),(10,15),(15,18)]
    if x.shape[0] != 19:
        raise ValueError
    else:
        montage = np.zeros((20, x.shape[1]))
        for idx, (i,j) in enumerate(montage_reference):
            montage[idx,:] = x[i,:] - x[j,:]
    return montage

FP1 - F7
F7 - T3
T3 - T5
T5 - O1
FP2 - F8
F8 - T4
T4 - T6
T6 - O2
T3 - C3
C3 - CZ
CZ - C4
C4 - T4
FP1 - F3
F3 - C3
C3 - P3
P3 - O1
FP2 - F4
F4 - C4
C4 - P4
P4 - O2


In [ ]:
import numpy as np
ntrain_preictal = 274
ntrain_interictal = 1647
ntest_preictal = 217
ntest_interictal = 775
nsample_interictal_train = int(np.floor(1647*ntest_preictal/ntest_interictal)*2)
sample_interictal_idx = np.random.choice(ntrain_interictal, nsample_interictal_train)
window_len_sec = 30
def window_eeg(x, window_sec, stride_sec, samp_freq):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    data_len_sec = npoint_original/samp_freq
    nwindow = np.floor((data_len_sec - window_sec)/stride_sec + 1)
    return nwindow
#     window_pt = int(samp_freq*window_sec)
#     windowed_eeg = np.zeros((nwindow,window_pt,nchannel))
#     for i in range(nwindow):
#         x_windowed = x[:,int(np.ceil(i*stride_sec*samp_freq)):int(np.ceil((i*stride_sec+window_sec)*samp_freq))]
#         #print(x_windowed.shape)
#         x_windowed = np.transpose(x_windowed)
#         windowed_eeg[i,:,:] = x_windowed
#     return windowed_eeg
def single_window_eeg(x, window_sec, samp_freq, position):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    data_len_sec = npoint_original/samp_freq
    window_pt = int(samp_freq*window_sec)
    windowed_eeg = np.zeros((window_pt,nchannel))
    if data_len_sec>window_sec:
        if position == 'end':
            x_windowed = x[:,npoint_original-1-window_pt:npoint_original-1]
        elif position == 'mid':
            midpoint = int(np.floor(npoint_original/2))
            halfwindow = int(np.floor(window_pt/2))
            #print(midpoint-halfwindow,midpoint+halfwindow+1)
            x_windowed = x[:,midpoint-halfwindow:midpoint+halfwindow]
        #print(x_windowed.shape)
        x_windowed = np.transpose(x_windowed)
        windowed_eeg[:,:] = x_windowed
        return True, windowed_eeg
    else:
        return False, windowed_eeg
def window_eeg_preictal(x,window_sec,stride_sec,samp_freq):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    data_len_sec = npoint_original/samp_freq
    window_pt = int(samp_freq*window_sec)
    #windowed_eeg = np.zeros((window_pt,nchannel))
    if data_len_sec > window_sec:
        nwindow = int(np.floor((data_len_sec - window_sec)/stride_sec + 1))
        #print(nwindow, data_len_sec)
        windowed_eeg = np.zeros((nwindow,window_pt,nchannel))
        for i in range(nwindow):
            x_windowed = x[:,int(np.ceil(i*stride_sec*samp_freq)):int(np.ceil((i*stride_sec+window_sec)*samp_freq))]
            x_windowed = np.transpose(x_windowed)
            windowed_eeg[i,:,:] = x_windowed
        return True, windowed_eeg
    else:
        return False, []    
all_train_x = []
all_train_y = []
n_valid_train_preictal = 0
n_valid_train_interictal = 0

for i in range(ntrain_preictal):
    train_preitctal_file_name = 'Processed_data_10s\\train\\TUH_preictal_{0}.npy'.format(str(i+1).zfill(3))
    raw = np.load(train_preitctal_file_name)
    valid, windowed = window_eeg_preictal(raw, window_len_sec, window_len_sec, 200)
    if valid:
        for j in range(windowed.shape[0]):
            all_train_x.append(windowed[j,:,:])
            all_train_y.append(1)
            n_valid_train_preictal += 1

print(n_valid_train_preictal)
            
nsample_interictal_train = int(np.floor(1647*ntest_preictal/ntest_interictal)*2)
sample_interictal_idx = np.random.choice(ntrain_interictal, nsample_interictal_train)

for i in range(ntrain_interictal):
    train_interictal_file_name = 'Processed_data_60s\\train\\TUH_interictal_{0}.npy'.format(str(i+1).zfill(3))
    raw = np.load(train_interictal_file_name)
    valid, windowed = single_window_eeg(raw, window_len_sec, 200, 'mid')
    if valid:
        all_train_x.append(windowed)
        all_train_y.append(0)
        n_valid_train_interictal += 1
print('number of training preictal data: ', n_valid_train_preictal, ', number of training interictal data: ', n_valid_train_interictal)
    
# train_interictal_file_name = 'Processed_data\train\TUH_interictal_{0}.npy'.format(str(i+1).zfill(3))
# test_preitctal_file_name = 'Processed_data\test\TUH_preictal_eval_{0}.npy'.format(str(i+1).zfill(3))
# test_interictal_file_name = 'Processed_data\test\TUH_interictal_eval_{0}.npy'.format(str(i+1).zfill(3))